In [1]:
print("Naveen kumar jadi")

naveen kumar jadi


In [8]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [7]:
# !pip install scikit-learn

In [9]:
# Set constants
FRAME_COUNT = 16  # Number of frames per video
IMAGE_SIZE = 112  # Resize frame to 112x112
BATCH_SIZE = 8
EPOCHS = 15
NUM_CLASSES = 4  # Only four classes: cricket, soccer, baseball, basketball
SPORTS_CLASSES = ["Cricket", "Soccer", "Baseball", "Basketball"]

# Path to dataset folder
DATASET_PATH = "/Users/naveenjadi/Desktop/naveen/naveen_experiments/videos"

In [10]:
# Function to extract frames from a video
def extract_frames(video_path, frame_count=FRAME_COUNT):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    interval = max(total_frames // frame_count, 1)  # Sample evenly across the video
    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
        frames.append(frame / 255.0)  # Normalize pixel values
    cap.release()

    # Pad with black frames if video has fewer frames
    while len(frames) < frame_count:
        frames.append(np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3)))
    return np.array(frames)


In [11]:
# Prepare dataset
def load_dataset(data_path, frame_count=FRAME_COUNT):
    videos = []
    labels = []
    class_map = {SPORTS_CLASSES[i]: i for i in range(len(SPORTS_CLASSES))}

    for cls_name in SPORTS_CLASSES:
        cls_path = os.path.join(data_path, cls_name)
        for video_file in os.listdir(cls_path):
            video_path = os.path.join(cls_path, video_file)
            frames = extract_frames(video_path, frame_count)
            videos.append(frames)
            labels.append(class_map[cls_name])

    # Convert to numpy arrays
    videos = np.array(videos)
    labels = to_categorical(labels, num_classes=len(SPORTS_CLASSES))
    return train_test_split(videos, labels, test_size=0.2, random_state=42)

# Load data
train_x, test_x, train_y, test_y = load_dataset(DATASET_PATH)


In [12]:
# Define 3D CNN Model
def build_3d_cnn(input_shape, num_classes):
    model = Sequential([
        Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Conv3D(64, (3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Conv3D(128, (3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [15]:
# Build and train the model
input_shape = (FRAME_COUNT, IMAGE_SIZE, IMAGE_SIZE, 3)
model = build_3d_cnn(input_shape, NUM_CLASSES)

history = model.fit(
    train_x, train_y,
    validation_split=0.2,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('/Users/naveenjadi/Desktop/naveen/naveen_experiments/model.keras', save_best_only=True, monitor='val_accuracy', mode='max'),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)


# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(f"Test Accuracy: {test_accuracy:.2f}")

Epoch 1/15


ValueError: Exception encountered when calling Conv3D.call().

[1mNegative dimension size caused by subtracting 3 from 2 for '{{node sequential_1_1/conv3d_5_1/convolution}} = Conv3D[T=DT_FLOAT, data_format="NDHWC", dilations=[1, 1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1, 1]](sequential_1_1/batch_normalization_4_1/batchnorm/add_1, sequential_1_1/conv3d_5_1/convolution/ReadVariableOp)' with input shapes: [?,2,26,26,64], [3,3,3,64,128].[0m

Arguments received by Conv3D.call():
  • inputs=tf.Tensor(shape=(None, 2, 26, 26, 64), dtype=float32)